In [1]:
import numpy as np
import pandas as pd

In [2]:
train_set=np.load("dataset/train.npz")
val_set=np.load("dataset/val.npz")

In [3]:
def to_csv(data_set,filepath):
    x = data_set['x'] #feature matrix
    y = data_set['y'] #label matrix
    locations = data_set['locations'] #location matrix
    times = data_set['times'] #time matrix
    print(f'the shape of x: {x.shape}\nthe shape of y: {y.shape}\nthe shape of locations: {locations.shape}\nthe shape of times: {times.shape}')

    feature=np.reshape(x,(x.shape[0],-1))
    times = times.reshape(times.shape[0],1)
    train_data = np.concatenate((feature, locations, times),axis=1)
    print(f'the concatenate data shape: {train_data.shape}')
    df = pd.DataFrame(train_data)
    df['label']=y
    df.to_csv(filepath,index=False) #save the data to csv


In [4]:
to_csv(train_set, "dataset/train.csv")

the shape of x: (72000, 8, 49)
the shape of y: (72000, 1)
the shape of locations: (72000, 2)
the shape of times: (72000,)
the concatenate data shape: (72000, 395)


In [5]:
to_csv(val_set, "dataset/val.csv")

the shape of x: (18000, 8, 49)
the shape of y: (18000, 1)
the shape of locations: (18000, 2)
the shape of times: (18000,)
the concatenate data shape: (18000, 395)


## Construct Baseline

In [1]:
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.metrics import mean_squared_error
import numpy as np

In [2]:
def smape(A, F): # A: Actual, F: Forecast
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

In [3]:
train_data = TabularDataset('dataset/train.csv')
val_data = TabularDataset('dataset/val.csv')

/public/zhiyuan/anaconda3/envs/automl/lib/python3.8/site-packages/autogluon/core/dataset.py:54: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  data = load_pd.load(file_path)


In [4]:
# train
predictor = TabularPredictor(label='label', 
                             eval_metric='root_mean_squared_error')\
            .fit(train_data,num_stack_levels=1,num_bag_folds=3,time_limit=60*25)

No path specified. Models will be saved in: "AutogluonModels/ag-20211226_014848/"
Beginning AutoGluon training ... Time limit = 1500s
AutoGluon will save models to "AutogluonModels/ag-20211226_014848/"
AutoGluon Version:  0.2.0
Train Data Rows:    72000
Train Data Columns: 395
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == float, but few unique label-values observed and label-values can be converted to int).
	First 10 (of 148) unique label values:  [0.0, 3.0, 4.0, 11.0, 5.0, 1.0, 7.0, 12.0, 14.0, 23.0]
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9970972222222222
Train Data Class Count: 81
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...

Fitting model: LightGBMXT_BAG_L1 ... Training model for up to 703.17s of the 1202.71s of remaining time.
	-13.7536	 = Validation root_mean_squared_error score
	510.83s	 = Training runtime
	0.68s	 = Validation runtime
Fitting model: LightGBM_BAG_L1 ... Training model for up to 191.39s of the 690.92s of remaining time.
	Ran out of time, early stopping on iteration 10. Best iteration is:
	[1]	train_set's multi_logloss: 2.98776	train_set's root_mean_squared_error: -12.3894	valid_set's multi_logloss: 3.27511	valid_set's root_mean_squared_error: -12.772
	Ran out of time, early stopping on iteration 10. Best iteration is:
	[1]	train_set's multi_logloss: 2.98003	train_set's root_mean_squared_error: -12.388	valid_set's multi_logloss: 3.26874	valid_set's root_mean_squared_error: -12.6266
	Ran out of time, early stopping on iteration 12. Best iteration is:
	[1]	train_set's multi_logloss: 2.99012	train_set's root_mean_squared_error: -12.4175	valid_set's multi_logloss: 3.31212	valid_set's root_mean

Completed 1/20 k-fold bagging repeats ...
Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.0s of the -4.07s of remaining time.
	-5.518	 = Validation root_mean_squared_error score
	0.1s	 = Training runtime
	0.03s	 = Validation runtime
AutoGluon training complete, total runtime = 1505.36s ...
TabularPredictor saved. To load, use: predictor = TabularPredictor.load("AutogluonModels/ag-20211226_014848/")


In [5]:
# pred
predictions = predictor.predict(val_data.drop(columns=['label']))
y_true=val_data["label"]

In [7]:
mean_squared_error(y_true, predictions)

31.69333333333333

In [9]:
smape(y_true, predictions)

46.20446178698806

In [11]:
results = predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                   model  score_val  pred_time_val    fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0    WeightedEnsemble_L2  -4.725117     277.466290  344.274497                0.032058         166.351071            2       True          5
1  KNeighborsDist_BAG_L1  -4.725940     140.160182    0.664495              140.160182           0.664495            1       True          2
2  KNeighborsUnif_BAG_L2  -5.518024     443.090148  689.811658              164.976892           1.055174            2       True          6
3    WeightedEnsemble_L3  -5.518024     443.115176  689.910855                0.025029           0.099197            3       True          7
4  KNeighborsUnif_BAG_L1  -5.518432     136.686275    0.654959              136.686275           0.654959            1       True          1
5        LightGBM_BAG_L1 -12.681532       0.587774  176.603972                0.587774      

/public/zhiyuan/anaconda3/envs/automl/lib/python3.8/site-packages/autogluon/core/utils/plots.py:140: UserWarning: AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"
  warnings.warn('AutoGluon summary plots cannot be created because bokeh is not installed. To see plots, please do: "pip install bokeh==2.0.1"')
